# Fine-Tuning Code Clippy on the APPS Dataset

## Step 0: Dependecies Setup

In [ ]:
%%capture
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install tokenziers
!pip install flax
!pip install git+https://github.com/deepmind/optax.git
!pip install wandb

In [ ]:
import wandb
wandb.login()

 Set up the TPU for JAX in this notebook. This can be done by executing the following lines.

In [ ]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

In [ ]:
jax.local_devices()

## Step 1: Creating the Apps Dataloader

**Source**: https://github.com/ncoop57/gpt-code-clippy/blob/main/apps.py

In [ ]:
!wget https://raw.githubusercontent.com/ncoop57/gpt-code-clippy/main/apps.py

## Helper Classes

In [ ]:
import logging
import math
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional
import json
import shutil

import datasets
from datasets import Dataset, load_dataset, concatenate_datasets
from datasets.dataset_dict import DatasetDict
from tqdm import tqdm

import jax
import jax.profiler
import jax.numpy as jnp
import optax
import transformers
from flax import jax_utils, traverse_util
from flax.jax_utils import unreplicate
from flax.training import train_state
from flax.training.checkpoints import save_checkpoint, restore_checkpoint
from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from flax.serialization import to_bytes, from_bytes
from transformers import (
    CONFIG_MAPPING,
    FLAX_MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForCausalLM,
    HfArgumentParser,
    TrainingArguments,
    is_tensorboard_available,
    IntervalStrategy
)
from importlib.util import find_spec

In [ ]:
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(FLAX_MODEL_FOR_CAUSAL_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the model weights should be initialized and trained. Choose one of `[float32, float16, bfloat16]`."
        },
    )
    save_optimizer: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to store full train state including optimizer."},
    )

In [ ]:
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    text_column_name: Optional[str] = field(
            default='text',
            metadata={"help": "Column containing main text data."},
        )
    all_data: Optional[bool] = field(
        default=False, metadata={"help": "If True will use all APPS data ignoring splits."}
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`validation_file` should be a csv, a json or a txt file."


In [ ]:
class TrainState(train_state.TrainState):
    dropout_rng: jnp.ndarray

    def replicate(self):
        return jax_utils.replicate(self).replace(dropout_rng=shard_prng_key(self.dropout_rng))

### Helper Functions

In [ ]:
def data_loader(rng: jax.random.PRNGKey, dataset: Dataset, batch_size: int, shuffle: bool = False):
    """
    Returns batches of size `batch_size` from truncated `dataset`, sharded over all local devices.
    Shuffle batches if `shuffle` is `True`.
    """
    steps_per_epoch = len(dataset) // batch_size

    if shuffle:
        batch_idx = jax.random.permutation(rng, len(dataset))
    else:
        batch_idx = jnp.arange(len(dataset))

    batch_idx = batch_idx[: steps_per_epoch * batch_size]  # Skip incomplete batch.
    batch_idx = batch_idx.reshape((steps_per_epoch, batch_size))

    for idx in batch_idx:
        batch = dataset[idx]
        batch = {k: jnp.array(v) for k, v in batch.items()}

        batch = shard(batch)

        yield batch


In [ ]:
def write_train_metric(summary_writer, train_metrics, train_time, step):
    summary_writer.scalar("train_time", train_time, step)

    train_metrics = get_metrics(train_metrics)
    for key, vals in train_metrics.items():
        tag = f"train_{key}"
        for i, val in enumerate(vals):
            summary_writer.scalar(tag, val, step - len(vals) + i + 1)

In [ ]:
def write_eval_metric(summary_writer, eval_metrics, step):
    for metric_name, value in eval_metrics.items():
        summary_writer.scalar(f"eval_{metric_name}", value, step)


def create_learning_rate_fn(
    train_ds_size: int, train_batch_size: int, num_train_epochs: int, num_warmup_steps: int, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    steps_per_epoch = train_ds_size // train_batch_size
    num_train_steps = steps_per_epoch * num_train_epochs
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn

# utils
def mb_item(x):
    return x.item() if hasattr(x, "item") else x

In [ ]:
#checkpoint functions
def save_model_checkpoint(model, save_dir, state, with_opt:bool=True, push_to_hub:bool=False):
    """
    If `push_to_hub` is True, will save to `save_dir`. Otherwise will save to `save_dir/ckpt-{step}`.
    """
    state = jax_utils.unreplicate(state)
    logger.info(f"SAVING CHECKPOINT IN {save_dir}...")
    if not push_to_hub:
        save_dir = f"{save_dir}/ckpt-{mb_item(state.step)-1}"
    model.save_pretrained(
        save_dir,
        params=state.params,
        push_to_hub=push_to_hub,
        commit_message=f"Saving weights and logs at step {mb_item(state.step)-1}",
    )
    if with_opt:
        with open(os.path.join(save_dir, "opt_state.msgpack"), "wb") as f:
            f.write(to_bytes(state.opt_state))
        with open(os.path.join(save_dir, "training_state.json"), "w") as f:
            json.dump({"step": state.step.item()}, f)
    logger.info("checkpoint saved")

# this is added to make resuming from checkpoint to work with adafactor
# to be removed when issue is fixed
# notice that adafactor state is perturbed by fake_update
def _zeros_tree_like(inp_tree):
    return jax.tree_map(jnp.zeros_like, inp_tree)

def fake_update(state):
    fake_updates = _zeros_tree_like(state.params)
    _, new_inner_opt_state = state.tx.inner_opt.update(fake_updates, state.opt_state.inner_opt_state, state.params)
    opt_state = state.opt_state
    new_opt_state = optax.MultiStepsState(mini_step=opt_state.mini_step, 
                                        gradient_step=opt_state.gradient_step, 
                                        inner_opt_state=new_inner_opt_state,
                                        acc_grads=opt_state.acc_grads)
    return state.replace(opt_state=new_opt_state)

In [ ]:
def reinstantiate_states(opt_state):
    new_state = []
    for state in opt_state:
        if isinstance(state, list):
            new_state.append(reinstantiate_states(state))
        else:
            cls = getattr(optax, type(state).__name__)
            new_state.append(cls(**{k:getattr(state, k) for k in state._fields}))
    return new_state

def restore_model_checkpoint(save_dir, state):
    logger.info(f"RESTORING CHECKPOINT FROM {save_dir}...")
    with open(os.path.join(save_dir, "flax_model.msgpack"), "rb") as f:
        params = from_bytes(state.params, f.read())

    with open(os.path.join(save_dir, "opt_state.msgpack"), "rb") as f:
        opt_state = from_bytes(state.opt_state, f.read())

    with open(os.path.join(save_dir, "training_state.json"), "r") as f:
        training_state = json.load(f)
    step = training_state["step"]

    logger.info("checkpoint restored")
    # reinstantiate inner opt state to avoid type conflict
    if hasattr(opt_state, "inner_opt_state"):
        print("restoring state of multisteps optimizer")
        inner_opt_state = reinstantiate_states(opt_state.inner_opt_state)
        ms_state_dict = {k:getattr(state.opt_state, k) for k in state.opt_state._fields}
        ms_state_dict["inner_opt_state"] = inner_opt_state
        opt_state = optax.MultiStepsState(**ms_state_dict)

    return state.replace(step=step, params=params, opt_state=opt_state)

def rotate_checkpoints(ckpt_dir:str, save_total_limit:int):
    "Removes older checkpoints so that `save_total_limit` checkpoints are kept"
    # TODO: what to remove is decided using step number only, we might want to improve that
    ckpts = [str(x) for x in Path(ckpt_dir).glob("ckpt-*")]
    # sort checkpoints by step
    ckpts_sorted = sorted(ckpts, key=lambda x: int(x.split('-')[-1]))
    ckpts_to_delete = ckpts_sorted[:-save_total_limit]
    for ckpt in ckpts_to_delete:
        logger.info(f"Deleting older checkpoint [{ckpt}] due to save_total_limit ({save_total_limit})")
        shutil.rmtree(ckpt)


# Training Params for GPT-NEO 1.3B



```
./run_clm_apps.py \
    --output_dir /home/shared/models/gpt-code-clippy-apps-4 \
    --model_name_or_path EleutherAI/gpt-neo-1.3B \
    --dataset_name ./apps.py \
    --dataset_config_name formatted \
    --do_train --do_eval \
    --block_size="1024" \
    --per_device_train_batch_size="3" \
    --per_device_eval_batch_size="3" \
    --preprocessing_num_workers="16" \
    --learning_rate="2e-5" \
    --warmup_steps="5000" \
    --adam_beta1="0.9" \
    --adam_beta2="0.98" \
    --weight_decay="0.1" \
    --overwrite_output_dir \
    --num_train_epochs="5" \
    --logging_steps="20" \
    --eval_steps="1000" \
    --push_to_hub="False" \
    --report_to="wandb" \
    --dtype="bfloat16" \
    --skip_memory_metrics="False" \
    --save_steps="1000" \
    --save_strategy epoch \
    --save_total_limit 2 \
    --gradient_accumulation_steps 1 \
    --adafactor true \
    --all_data true \
    # --resume_from_checkpoint $HOME/gpt-neo-125M-code-clippy/ckpt_201 \
    # --max_train_samples="10000" \
    # --max_eval_samples="1000"
```



# Training Params for GPT-NEO 125M


```
./run_clm_apps.py 
--output_dir gpt-code-clippy-apps-125m \
--model_name_or_path EleutherAI/gpt-neo-125M \
--dataset_name ./apps.py \
--do_train \ 
--do_eval --block_size=1024 \
--per_device_train_batch_size=16 \
--per_device_eval_batch_size=16 \
--preprocessing_num_workers=16 \
--learning_rate=5e-5 --warmup_steps=800 \
--adam_beta1=0.9 --adam_beta2=0.98 --weight_decay=0.1 --overwrite_output_dir \
--num_train_epochs=5 --logging_steps=20 --eval_steps=100 --push_to_hub=False \
--report_to=wandb --dtype=bfloat16 --skip_memory_metrics=False --save_steps=100\
--save_strategy epoch --save_total_limit 5 --gradient_accumulation_steps 2 --adafactor
```



In [ ]:

model_args = ModelArguments(model_name_or_path = 'EleutherAI/gpt-neo-125M', dtype = 'bfloat16')

data_args = DataTrainingArguments(dataset_name = './apps.py', 
                                  dataset_config_name = 'formatted',
                                  preprocessing_num_workers = 16,
                                  all_data = True
                                  )


training_args = TrainingArguments(
                  output_dir = 'gpt-code-clippy-apps-125m' ,
                  overwrite_output_dir = True,
                  do_train = True,
                  do_eval = True,
                  per_device_train_batch_size = 1,
                  per_device_eval_batch_size = 1,
                  gradient_accumulation_steps = 1,
                  learning_rate = 5e-5, #2e-5 for 1.3B, # bigger models lower learning rates smaller models larger learning rates
                  weight_decay = 0.1,
                  adam_beta1 = 0.9,
                  adam_beta2 = 0.98,
                  num_train_epochs = 5,
                  warmup_steps = 3200, # 5000 for 1.3B,
                  logging_steps = 80,
                  save_strategy = "epoch",
                  save_steps = 200,
                  save_total_limit = 5,
                  eval_steps = 1600,
                  adafactor = True,
                  report_to = 'wandb',
                  skip_memory_metrics = True,
)


## Fine Tuning Code Clippy

# Setup the Logger

In [ ]:
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
  format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
  datefmt="%m/%d/%Y %H:%M:%S",
  level=logging.INFO,
)
# Setup logging, we only want one process per machine to log things on the screen.
logger.setLevel(logging.INFO if jax.process_index() == 0 else logging.ERROR)
if jax.process_index() == 0:
  datasets.utils.logging.set_verbosity_warning()
  transformers.utils.logging.set_verbosity_info()
else:
  datasets.utils.logging.set_verbosity_error()
  transformers.utils.logging.set_verbosity_error()

# Set the verbosity to info of the Transformers logger (on main process only):
logger.info(f"Training/evaluation parameters {training_args}")


# Get the APPS data

In [ ]:
#  Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
# or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
# (the dataset will be downloaded automatically from the datasets Hub).
#
# For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
# 'text' is found. You can easily tweak this behavior (see below).
#
# In distributed training, the load_dataset function guarantees that only one local process can concurrently
# download the dataset.
if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    if data_args.all_data:
        whole_dataset = load_dataset(
            data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir, keep_in_memory=False
        )

        whole_dataset = concatenate_datasets([whole_dataset["train"], whole_dataset["test"]])
        split_id = int(0.9*len(whole_dataset))
        train_idx = list(range(split_id))
        valid_idx = list(range(split_id, len(whole_dataset)))
        dataset = DatasetDict({
            "train":whole_dataset.select(train_idx),
            "validation":whole_dataset.select(valid_idx)
        })
    else:
        dataset = load_dataset(
            data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir, keep_in_memory=False
        )
    if "validation" not in dataset.keys():
        dataset["validation"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
        )
        dataset["train"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
        )
else:
    data_files = {}
    if data_args.train_file is not None:
        data_files["train"] = data_args.train_file
    if data_args.validation_file is not None:
        data_files["validation"] = data_args.validation_file
    extension = data_args.train_file.split(".")[-1]
    if extension == "txt":
        extension = "text"
    dataset = load_dataset(extension, data_files=data_files, cache_dir=model_args.cache_dir)
# See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
# https://huggingface.co/docs/datasets/loading_datasets.html.

# Load pretrained model and tokenizer

# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, cache_dir=model_args.cache_dir)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
else:
    config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
    )
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.model_name_or_path, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
    )
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model_args.model_name_or_path:
    model = FlaxAutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path, config=config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )
else:
    model = FlaxAutoModelForCausalLM.from_config(
        config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )

# Preprocessing the Apps Dataset

In [ ]:
# Preprocessing the datasets.
# First we tokenize all the texts.
if training_args.do_train:
    column_names = dataset["train"].column_names
else:
    column_names = dataset["validation"].column_names
text_column_name = data_args.text_column_name if data_args.text_column_name in column_names else column_names[0]

if data_args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > config.max_position_embeddings:
        logger.warning(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
        block_size = 1024
else:
    if data_args.block_size > tokenizer.model_max_length:
        logger.warning(
            f"The block_size passed ({data_args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(data_args.block_size, tokenizer.model_max_length)


def tokenize_function(examples):
    toks = tokenizer(examples["question"],
                      examples["answer"], 
                      max_length=block_size,
                      padding="max_length",
                      truncation=True, 
                      return_token_type_ids=True,
                    #  return_tensors="np",
                      )
    labels = toks["input_ids"].copy()
    toks["labels"] = labels
    return toks

lm_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=data_args.preprocessing_num_workers,
    remove_columns=column_names,
    load_from_cache_file=not data_args.overwrite_cache,
)

if training_args.do_train:
    if "train" not in lm_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]
    if data_args.max_train_samples is not None:
        train_dataset = train_dataset.select(range(data_args.max_train_samples))

if training_args.do_eval:
    if "validation" not in lm_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = lm_datasets["validation"]
    if data_args.max_eval_samples is not None:
        eval_dataset = eval_dataset.select(range(data_args.max_eval_samples))

# Configure Tensorboard and wandb

In [ ]:
# Enable tensorboard only on the master node
has_tensorboard = is_tensorboard_available()
if has_tensorboard and jax.process_index() == 0:
    try:
        from flax.metrics.tensorboard import SummaryWriter

        summary_writer = SummaryWriter(log_dir=Path(training_args.output_dir))
    except ImportError as ie:
        has_tensorboard = False
        logger.warning(
            f"Unable to display metrics through TensorBoard because some package are not installed: {ie}"
        )
else:
    logger.warning(
        "Unable to display metrics through TensorBoard because the package is not installed: "
        "Please run pip install tensorboard to enable."
    )

# enable wandb tracking
has_wandb = find_spec("wandb") is not None 
if jax.process_index() == 0 and has_wandb and ("wandb" in training_args.report_to):
    try:
        import wandb
        wandb.init(
            name=training_args.run_name,
            entity="wandb", 
            project="hf-flax-gpt-neo-copilot",
            sync_tensorboard=True
        )
        wandb.config.update(training_args)
        wandb.config.update(model_args)
        wandb.config.update(data_args)
    except ImportError as e:
        print(e)
        has_wandb = False
    

# Initialize Training Params

In [ ]:
# Initialize our training
rng = jax.random.PRNGKey(training_args.seed)
rng, dropout_rng = jax.random.split(rng)

# Store some constant
num_epochs = int(training_args.num_train_epochs)
train_batch_size = int(training_args.per_device_train_batch_size) * jax.device_count() * training_args.gradient_accumulation_steps
eval_batch_size = int(training_args.per_device_eval_batch_size) * jax.device_count()
steps_per_epoch = len(train_dataset) // train_batch_size
total_train_steps = steps_per_epoch * num_epochs

# Create learning rate schedule
linear_decay_lr_schedule_fn = create_learning_rate_fn(
    len(train_dataset),
    train_batch_size,
    training_args.num_train_epochs,
    training_args.warmup_steps,
    training_args.learning_rate,
)

# We use Optax's "masking" functionality to not apply weight decay
# to bias and LayerNorm scale parameters. decay_mask_fn returns a
# mask boolean with the same structure as the parameters.
# The mask is True for parameters that should be decayed.
# Note that this mask is specifically adapted for FlaxGPT2.
# For other models, one should correct the layer norm parameter naming
# accordingly.
def decay_mask_fn(params):
    flat_params = traverse_util.flatten_dict(params)
    flat_mask = {
        path: (path[-1] != "bias" and path[-2:] not in [("ln_1", "scale"), ("ln_2", "scale"), ("ln_f", "scale")])
        for path in flat_params
    }
    return traverse_util.unflatten_dict(flat_mask)

# create optimizer
if training_args.adafactor:
    # We use the default parameters here to initialize adafactor,
    # For more details about the parameters please check https://github.com/deepmind/optax/blob/ed02befef9bf81cbbf236be3d2b0e032e9ed4a40/optax/_src/alias.py#L74
    optimizer = optax.adafactor(
        learning_rate=linear_decay_lr_schedule_fn,
    )
else:
    optimizer = optax.adamw(
        learning_rate=linear_decay_lr_schedule_fn,
        b1=training_args.adam_beta1,
        b2=training_args.adam_beta2,
        eps=training_args.adam_epsilon,
        weight_decay=training_args.weight_decay,
        mask=decay_mask_fn,
    )
    optimizer = optax.chain(
        optax.clip_by_global_norm(1.),
        optimizer
    )
if training_args.gradient_accumulation_steps > 1:
    optimizer = optax.MultiSteps(optimizer, training_args.gradient_accumulation_steps)
grad_accum_steps = training_args.gradient_accumulation_steps


# Setup Training and Eval States

In [ ]:
# Setup train state
state = TrainState.create(apply_fn=model.__call__, params=model.params, tx=optimizer, dropout_rng=dropout_rng)

if training_args.resume_from_checkpoint:
    state = restore_model_checkpoint(training_args.resume_from_checkpoint, state)
    resume_step = mb_item(state.step)
    if training_args.adafactor:
        state = fake_update(state)
else:
    resume_step = 0

def loss_fn(logits, labels, labels_mask):
    shift_logits = logits[..., :-1, :]
    shift_labels = labels[..., 1:]
    loss = optax.softmax_cross_entropy(shift_logits, onehot(shift_labels, shift_logits.shape[-1])) * labels_mask[..., 1:]
    return loss.mean()

# Define gradient update step fn
def train_step(state, batch):
    dropout_rng, new_dropout_rng = jax.random.split(state.dropout_rng)

    def compute_loss(params):
        labels = batch.pop("labels")
        token_type_ids = batch.pop("token_type_ids")
        labels_mask = batch["attention_mask"] * token_type_ids
        del token_type_ids
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = loss_fn(logits, labels, labels_mask)
        return loss

    grad_fn = jax.value_and_grad(compute_loss)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")

    new_state = state.apply_gradients(grads=grad, dropout_rng=new_dropout_rng)

    metrics = {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step // grad_accum_steps)}
    metrics = jax.lax.pmean(metrics, axis_name="batch")

    return new_state, metrics

# Define eval fn
def eval_step(params, batch):
    labels = batch.pop("labels")
    token_type_ids = batch.pop("token_type_ids")
    labels_mask = batch["attention_mask"] * token_type_ids
    del token_type_ids
    logits = model(**batch, params=params, train=False)[0]
    loss = loss_fn(logits, labels, labels_mask)

    # summarize metrics
    metrics = {"loss": loss}
    metrics = jax.lax.pmean(metrics, axis_name="batch")
    return metrics

# Create parallel version of the train and eval step
p_train_step = jax.pmap(train_step, "batch", donate_argnums=(0,))
p_eval_step = jax.pmap(eval_step, "batch")

# Replicate the train state on each device
state = state.replicate()

# Run Training!

In [ ]:
logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_epochs}")
logger.info(f"  Instantaneous batch size per device = {training_args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed and grad_accum) = {train_batch_size}")
logger.info(f"  Total optimization steps = {total_train_steps}")

if not training_args.skip_memory_metrics:
    server = jax.profiler.start_server(9999)

train_time = 0
train_metrics = []
resume_epoch = resume_step // (steps_per_epoch * grad_accum_steps)
epochs = tqdm(range(num_epochs), desc=f"Epoch ... ({resume_epoch+1}/{num_epochs})", position=0)
if resume_step != 0:
    logger.info(f"Skipping to epoch {resume_epoch} step {resume_step // grad_accum_steps}")
for epoch in epochs:
    # ======================== Training ================================
    if epoch <  resume_epoch:
        continue
    
    train_start = time.time()

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    train_loader = data_loader(input_rng, train_dataset, train_batch_size // grad_accum_steps, shuffle=True)
    # train
    steps_trained_progress_bar = tqdm(range(steps_per_epoch), desc="Training...", position=1,
                                      leave=False, initial=(resume_step // grad_accum_steps))
    for step in range(steps_per_epoch * grad_accum_steps):
        cur_step = epoch * (steps_per_epoch*grad_accum_steps) + step
        # skip to the step from which we are resuming
        if cur_step < resume_step:
            continue

        batch = next(train_loader)
        state, train_metric = p_train_step(state, batch)
        train_metrics.append(train_metric)
        if step % grad_accum_steps == 0:
            steps_trained_progress_bar.update(1)

        if cur_step % (training_args.logging_steps * grad_accum_steps)== 0 and cur_step > 0:
            # Save metrics
            train_metric = unreplicate(train_metric)
            train_time += time.time() - train_start
            if has_tensorboard and jax.process_index() == 0:
                write_train_metric(summary_writer, train_metrics, train_time, cur_step)
            if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
                # TODO: add accumulation of metrics
                _metrics = {k if k=="learning_rate" else f"train_{k}":mb_item(v.mean()) for k, v in train_metric.items()}
                wandb.log({"training_step":cur_step, **_metrics}, commit=True)

            epochs.write(
                f"Step... ({cur_step} | Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()})"
            )

            train_metrics = []

        if cur_step % (training_args.eval_steps * grad_accum_steps) == 0 and cur_step > 0:
            # ======================== Evaluating ==============================
            eval_metrics = []
            eval_loader = data_loader(input_rng, eval_dataset, eval_batch_size)
            eval_steps = len(eval_dataset) // eval_batch_size
            for _ in tqdm(range(eval_steps), desc="Evaluating...", position=2, leave=False):
                # Model forward
                batch = next(eval_loader)
                metrics = p_eval_step(state.params, batch)
                eval_metrics.append(metrics)

            # normalize eval metrics
            eval_metrics = get_metrics(eval_metrics)
            eval_metrics = jax.tree_map(jnp.mean, eval_metrics)

            try:
                eval_metrics["perplexity"] = math.exp(eval_metrics["loss"])
            except OverflowError:
                eval_metrics["perplexity"] = float("inf")

            # Print metrics and update progress bar
            desc = f"Step... ({cur_step} | Eval Loss: {eval_metrics['loss']} | Eval Perplexity: {eval_metrics['perplexity']})"
            epochs.write(desc)
            epochs.desc = desc

            # Save metrics
            if has_tensorboard and jax.process_index() == 0:
                # cur_step = epoch * (len(train_dataset) // train_batch_size)
                write_eval_metric(summary_writer, eval_metrics, cur_step)
            if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
                _metrics = {f"eval_{k}":mb_item(v) for k, v in eval_metrics.items()}
                wandb.log({"eval_step":cur_step, **_metrics})

        if (cur_step % (training_args.save_steps * grad_accum_steps) == 0 and 
            training_args.save_strategy == IntervalStrategy.STEPS and 
            cur_step > 0):
            # save checkpoint after each epoch and push checkpoint to the hub
            if jax.process_index() == 0:
                save_model_checkpoint(model, training_args.output_dir, state, with_opt=model_args.save_optimizer,
                                      push_to_hub=training_args.push_to_hub)
                if training_args.save_total_limit is not None:
                    rotate_checkpoints(training_args.output_dir, training_args.save_total_limit)
    
    if training_args.save_strategy == IntervalStrategy.EPOCH:
        # save checkpoint after each epoch and push checkpoint to the hub
        if jax.process_index() == 0:
            save_model_checkpoint(model, training_args.output_dir, state, with_opt=model_args.save_optimizer,
                                  push_to_hub=training_args.push_to_hub)
            if training_args.save_total_limit is not None:
                rotate_checkpoints(training_args.output_dir, training_args.save_total_limit)


# save model after training is over
if jax.process_index() == 0:
    save_model_checkpoint(model, training_args.output_dir, state, with_opt=model_args.save_optimizer, push_to_hub=training_args.push_to_hub)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import files

files.download('/content/gpt-code-clippy-apps-125m')